In [ ]:
import os
import json
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12
plt.rcParams["font.family"] = "Times New Roman"

''' Install fonts on Ubuntu:
sudo apt install msttcorefonts -qq
rm ~/.cache/matplotlib -rf           # remove cache
'''

In [ ]:
def extract_plot_data(results):
    plot_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    xrange = np.array(list(results.keys())).astype(int)
    for num_sensors in results.keys():
        for method in  results[num_sensors].keys():
            for metric in results[num_sensors][method].keys():
                data = np.array(results[num_sensors][method][metric])
                if 'Time' in metric and 'Adaptive' in method:
                    data /= int(num_sensors)
                plot_data[metric][method]['mean'].append(np.mean(data))
                plot_data[metric][method]['std'].append(np.std(data))
    return xrange, plot_data

def plot_metric(xrange, data, xlabel, ylabel,
                colors=None, methods=None, 
                labels=None, fname=None):
    if methods is None:
        methods = list(data.keys())
    if labels is None:
        labels = methods
    if colors is None:
        colors = [None]*len(methods)

    plt.figure(figsize=(10, 5))
    for i, key in enumerate(methods):
        xrange_ = xrange[:len(data[key]['mean'])]
        label = labels[i]

        plt.plot(xrange_, data[key]['mean'], label=label)
        plt.fill_between(xrange_,
                         np.array(data[key]['mean']) + np.array(data[key]['std']),
                         np.array(data[key]['mean']) - np.array(data[key]['std']),
                         alpha=0.2)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.tight_layout()
    if fname is not None:
        plt.savefig(f'{fname}', bbox_inches='tight', dpi=500)
    plt.show()

In [ ]:
filename = f'IPP-mississippi_1R_2S.json'
print(f'Loading results from {filename}')
save_filename = filename[:-5] + '-{}.pdf'
save_filename = os.path.join('plots', save_filename)
if not os.path.exists('plots'):
    os.makedirs('plots')
results = json.load(open(filename, 'r'))
xrange, plot_data = extract_plot_data(results)
print(f'Done!')

In [ ]:
methods = plot_data['RMSE'].keys()
#methods = [m for m in methods if "CMA" not in m]
methods = [m for m in methods if "Diff" not in m]
methods = [m for m in methods if "Greedy" not in m]
    
plot_metric(xrange, plot_data['IPPTime'], "Number of Waypoints", "Runtime (s)", methods=methods)
plot_metric(xrange, plot_data['NLPD'], "Number of Waypoints", "SMSE", methods=methods)